In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('players_20.csv')
df.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,24-06-1987,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,01-07-2004,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,"Beat Offside Trap, Argues with Officials, Earl...",88,95,70,92,88,97,93,94,92,96,91,84,93,95,95,86,68,75,68,94,48,40,94,94,75,96,33,37,26,6,11,15,14,8
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,05-02-1985,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,10-07-2018,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,"Long Throw-in, Selfish, Argues with Officials,...",84,94,89,83,87,89,81,76,77,92,89,91,87,96,71,95,95,85,78,93,63,29,95,82,85,95,28,32,24,7,11,15,14,11
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,05-02-1992,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,03-08-2017,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,"Power Free-Kick, Injury Free, Selfish, Early C...",87,87,62,87,87,96,88,87,81,95,94,89,96,92,84,80,61,81,49,84,51,36,87,90,90,94,27,26,29,9,9,15,15,11
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,07-01-1993,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,16-07-2014,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"Flair, Acrobatic Clearance",13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,27,12,18,87,92,78,90,89
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,07-01-1991,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,01-07-2019,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,"Beat Offside Trap, Selfish, Finesse Shot, Spee...",81,84,61,89,83,95,83,79,83,94,94,88,95,90,94,82,56,84,63,80,54,41,87,89,88,91,34,27,22,11,12,6,8,8


In [ ]:
df.shape

(18278, 72)

In [ ]:
# replacing null values with 0
df = df.fillna(value= 0)

Euclidean distance

In [ ]:
X=df['overall']
Y=df['potential']

sum1=0 # sum of squares of all the values of X
sum2=0 # sum of squares of all the values of Y
sum3=0 # sum of squares of all the 2*X*Y values

for i in X:
  sum1+=i*i

for j in Y:
  sum2+=j*j 

for (i,j) in zip(X,Y):
  sum3+=2*i*j

euclidean_dist=(sum1+sum2-sum3)**0.5
print("Euclidean distance between attributes overall and potential is: ",euclidean_dist)

Euclidean distance between attributes overall and potential is:  1037.7581606520857


Manhattan distance

In [ ]:
X=df['overall']
Y=df['potential']

manhattan_dist=0

for (i,j) in zip(X,Y):
  manhattan_dist+=abs(i-j)


print("Manhattan distance between attributes overall and potential is: ",manhattan_dist)

Manhattan distance between attributes overall and potential is:  96908


Supremum distance

In [ ]:
X=df['overall']
Y=df['potential']

supremum_dist=0

for (i,j) in zip(X,Y):
  if abs(i-j)>supremum_dist:
    supremum_dist=abs(i-j)


print("Supremum distance between attributes overall and potential is: ",supremum_dist)

Supremum distance between attributes overall and potential is:  25


In [ ]:
#confirming using the inbuilt functions
from scipy.spatial import distance
print("euclidean",distance.minkowski(X, Y, 2))
print("manhattan",distance.minkowski(X, Y, 1))
print("supremum",distance.minkowski(X, Y, float('inf')))

euclidean 1037.7581606520857
manhattan 96908.0
supremum 25.0


Mahalanobis distance

In [ ]:
g1=df[['overall','potential']][:9131]
g2=df[['overall','potential']][9131:]

# Calculating the mean of each of the attribute from each group

sum_g1_1=0
sum_g1_2=0
sum_g2_1=0
sum_g2_2=0

for i in g1['overall']:
  sum_g1_1+=i
for i in g1['potential']:
  sum_g1_2+=i

for i in g2['overall']:
  sum_g2_1+=i
for i in g2['potential']:
  sum_g2_2+=i

mean_g1=[round(sum_g1_1/len(g1),2),round(sum_g1_2/len(g1),2)]
mean_g2=[round(sum_g2_1/len(g2),2),round(sum_g2_2/len(g2),2)]

print("The mean of the attributes in group 1 is: ",mean_g1)
print("The mean of the attributes in group 2 is: ",mean_g2)

The mean of the attributes in group 1 is:  [71.72, 74.83]
The mean of the attributes in group 2 is:  [60.78, 68.27]


In [ ]:
# Calculating the mean difference vector

mean_diff=[]

for (i,j) in zip(mean_g1,mean_g2):
  mean_diff.append(round(i-j,2))

print(mean_diff)

[10.94, 6.56]


In [ ]:
# Calculating the covariance matrix for both the groups
import itertools 

def covariance_matrix(group,mean):
  covar_matrix=[]
  for i,j in itertools.product(range(group.shape[1]),range(group.shape[1])):
    temp=[]
    for m,n in zip(group.iloc[:,i],group.iloc[:,j]):
      temp.append(round(((m-mean[i])*(n-mean[j])),4))
    covar_matrix.append(sum(temp)/(len(group)-1))

  covar_matrix=np.array(covar_matrix).reshape((2,2))
  return covar_matrix

print("Covariance matrix of group1: ",covariance_matrix(g1,mean_g1))
print("")
print("Covariance matrix of group2: ",covariance_matrix(g2,mean_g2))

Covariance matrix of group1:  [[19.06744692 15.19175417]
 [15.19175417 26.59795245]]

Covariance matrix of group2:  [[17.71676304  4.16510149]
 [ 4.16510149 27.32416535]]


In [ ]:
# Calculating the common(pooled)covariance matrix from two covariance matrix computed in earlier step

common_covar=(len(g1)/(len(g1)+len(g2)))*covariance_matrix(g1,mean_g1) + (len(g2)/(len(g1)+len(g2)))*covariance_matrix(g2,mean_g2)
print("Common covariance matrix: ",common_covar)

Common covariance matrix:  [[18.39151381  9.67360163]
 [ 9.67360163 26.96137675]]


In [ ]:
# Calculating the inverse covariance matrix of common(pooled) covariance matrix using inbuilt function

common_covar_inverse=np.linalg.inv(common_covar)
print(common_covar_inverse)

[[ 0.06702109 -0.02404682]
 [-0.02404682  0.04571797]]


In [ ]:
def transposeMatrix(m):
    return map(list,zip(*m))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return determinant

def getMatrixInverse(m):
    determinant = getMatrixDeternminant(m)
    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return cofactors

common_covar_inverse=getMatrixInverse(common_covar)
print(common_covar_inverse)

[[0.06702109252811617, -0.02404681912419998], [-0.02404681912419998, 0.0457179675943323]]


In [ ]:
# Calculating the transpose of the mean difference vector 
print(np.transpose(mean_diff))

[10.94  6.56]


In [ ]:
# Calculating Mahalonobis square distance using the formula
m=np.mean(np.multiply(mean_diff,np.multiply(common_covar_inverse,mean_diff)))
print("mahalanobis squarred distance matrix: ",np.multiply(np.transpose(mean_diff),np.multiply(common_covar_inverse,mean_diff)))
print("")
print("mahalanobis squarred distance: ",m)

mahalanobis squarred distance matrix:  [[ 8.02132563 -1.0348212 ]
 [-2.87800988  1.96740873]]

mahalanobis squarred distance:  1.5189758207923068
